# Setup

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import scanpy as sc
import scrnatools as rna # custom helper methods for scRNAseq analysis available at https://github.com/j-germino/sc-rna-tools-git.git
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import scvi

In [ ]:
logger = rna.configs.create_logger(__name__)
rna.configs.verbosity = "warning"
sns.set_theme(context="paper", style="white", )
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["savefig.facecolor"] = "white"
# Set working directory to project root folder
if os.getcwd() != "I:\Desktop\Joe\Projects\eTAC-analysis":
    os.chdir("I:\Desktop\Joe\Projects\eTAC-analysis")

In [ ]:
os.getcwd()

# Import data

In [ ]:
# Cellranger h5 files in 'data' subfolder

# New sort data
wtPos = sc.read_10x_h5("data/uncombined_WT_KO_data/WT_Pos_fiveprime_rna_filtered_feature_bc_matrix.h5")
wtPos.var_names_make_unique()
wtPos.obs["mouse"] = "WT"
wtPos.obs["sort"] = "GFPp"
wtPos.obs["sample"] = "new data"
wtPos.layers["raw_counts"] = wtPos.X.copy()

wtRef = sc.read_10x_h5("data/uncombined_WT_KO_data/WT_Ref_fiveprime_rna_filtered_feature_bc_matrix.h5")
wtRef.var_names_make_unique()
wtRef.obs["mouse"] = "WT"
wtRef.obs["sort"] = "unsorted"
wtRef.obs["sample"] = "new data"
wtRef.layers["raw_counts"] = wtRef.X.copy()

# Published data (Wang et al, Science Immunol. 2021) DOI: 10.1126/sciimmunol.abl5053
wtPosOld = sc.read_10x_h5("data/eTAC_old_data/RNA_GFP_filtered_feature_bc_matrix.h5")
wtPosOld.var_names_make_unique()
wtPosOld.obs["mouse"] = "WT"
wtPosOld.obs["sort"] = "GFPp"
wtPosOld.obs["sample"] = "Wang, et al. 2021"
wtPosOld.layers["raw_counts"] = wtPosOld.X.copy()

wtRefOld = sc.read_10x_h5("data/eTAC_old_data/RNA_WT_filtered_feature_bc_matrix.h5")
wtRefOld.var_names_make_unique()
wtRefOld.obs["mouse"] = "WT"
wtRefOld.obs["sort"] = "unsorted"
wtRefOld.obs["sample"] = "Wang, et al. 2021"
wtRefOld.layers["raw_counts"] = wtRefOld.X.copy()

eTACdict = {"WT_pos": wtPos, "WT_ref": wtRef, "WT_pos_old": wtPosOld, "WT_ref_old": wtRefOld}

# Doublet filter

In [ ]:
# Filter doublets using scrublet
eTACdict = rna.qc.doublet_filter(
    eTACdict,
    score_threshold=0.3,
    save_path="analysis/qc/scrublet/WT_LN_merged/",
    dpi=300,
    context="paper",
)

# Merge Data

In [ ]:
lnData = rna.tl.merge_anndata_dict(eTACdict, join="inner")
lnData.obs["batch_key"] = lnData.obs.batch # Batch key is each individual sample (old v new + sort condition)
lnData.write("processed_data/h5ad_files/WT_LN_merged_raw.h5ad")

# QC

In [ ]:
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_raw.h5ad")
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, percent_top=None, qc_vars=["mt"], log1p=False, inplace=True)
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_counts=1)
logger.info(f"Num cells before filter: {len(adata.obs)}")
adata = rna.qc.filter_cells(
    adata,
    gene_thresholds=(600,5000),
    count_thresholds=(1000,30000),
    save_path="analysis/qc/WT_LN_merged",
    hue_key="batch",
    title="WT LN combined",
    dpi=300,
    context="paper",
    dims=(18,5),
    s=3,
)
logger.info(f"Num cells after filter: {len(adata.obs)}")

# Preprocessing

In [ ]:
logger.info("Normalizing and log-transforming counts")
# Normalize counts per cell
sc.pp.normalize_total(adata, target_sum=1e4)
# Log-transform counts
sc.pp.log1p(adata)

# Save raw, log transformed counts
adata.raw = adata

logger.info("Regress out ['total_counts', 'pct_counts_mt'] and scale")
# Regress and scale all genes
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
sc.pp.scale(adata, max_value=10)

# Save the preprocessed data for scVI training
adata.write("processed_data/h5ad_files/WT_LN_merged_preprocessed.h5ad")

# scVI training

In [ ]:
# Make sure path for scVI models exists
if not os.path.exists("processed_data/scVI_models/"):
        os.makedirs("processed_data/scVI_models/")

# Train model using raw counts, batch key as sample + sort combination (req. GPU)
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_preprocessed.h5ad")
scvi.model.SCVI.setup_anndata(adata, layer="raw_counts", batch_key="batch_key")
model = scvi.model.SCVI(adata)
model.train(use_gpu=True)
model.save("processed_data/scVI_models/WT_LN_merged")

# Analysis

## Make UMAP, tSNE embeddings, get normalized expression

In [ ]:
# Work from files saved for scVI training
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_preprocessed.h5ad")
model = scvi.model.SCVI.load(
    "processed_data/scVI_models/WT_LN_merged",
    adata
)
# Get latent representation
adata.obsm["X_scVI"] = model.get_latent_representation()
# Save scVI normalized expression to a new layer, normalized to 10e4 counts per cell
adata.layers["scVI_normalized"] = model.get_normalized_expression(library_size=10e4)
# Calculate neighbors using scVI latent space
sc.pp.neighbors(adata, use_rep="X_scVI")
# Calculate UMAP, tSNE coordinates for each cell
sc.tl.umap(adata, min_dist=0.5, spread=1)
sc.tl.tsne(adata, use_rep="X_scVI")
# Save processed data as a new h5ad file
adata.write("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")

## Leiden clustering

In [ ]:
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")
sc.tl.leiden(adata, key_added="leiden", resolution=0.8)
adata.write("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")

## Cell type annotation

In [ ]:
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")
adata.obs["cell_type"] = adata.obs.leiden
# Remove noise/junk cell clusters
adata = adata[~adata.obs.cell_type.isin(["23", "24", "29"])].copy()
# Relabel leiden cluster numbers as cell types
cellTypes = {
    "18": "JC2",
    "5": "JC1",
    "15": "JC3",
    "8": "mDC2-5",
    "2": "mDC2-4",
    "4": "mDC2-3",
    "13": "mDC2-2",
    "1": "mDC2-1",
    "0": "mDC1-2",
    "3": "mDC1-1",
    "11": "mDC1-3",
    "17": "ILC3",
    "12": "CD4 T",
    "14": "CD8 T",
    "16": "B",
    "20": "Fibro1",
    "28": "Fibro3",
    "27": "Fibro2",
    "6": "MF",
    "9": "cDC2",
    "10": "cDC1",
    "21": "Plasma cell",
    "25": "pDC",
    "19": "Mo",
    "7": "BEC",
    "26": "ILC",
    "22": "LEC",
}
adata.obs.cell_type = adata.obs.cell_type.replace(cellTypes)

# Feature plots for cell type annotation
rna.pl.sc_umap(
    adata,
    ["cell_type", "sort", "Aire", "H2-Aa", "Ccr7", "Rorc", "Irf4", "Irf8", "Ptprc", "Cd8a", "Cd4", "Cd3e", 'Cd19', "Itgae", "Col1a1", "Loxl1", "Lum", "Fbln1", "Siglech", "Cd79a", 
     "Mki67", "Cd69","Igha", "Xcr1","Cd207","Pld4", "Ccr9", "Sirpa", "Clec4n", "Il1b","Cd34", "Adgre1", "Itgax", "Bst2", "Ly6c1", "Cd14", "Cd68", "Ccr5", "Il7r", "Sdc1", "Itgam"],
    legend_loc=["on_data", "bottom"],
    cmaps=["gist_rainbow", "tab10", "viridis"],
    gene_data="scVI_normalized",
    latent_rep="X_tsne",
    context="paper",
    dims=(6, 6),
    s=5,
    save_path="analysis/feature_plots/WT_LN_cell_type_features.png",
    dpi=300,
    on_data_text_size=10,
    legend_size=10,
)
adata.write("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")

## DE genes all clusters

In [ ]:
adata = sc.read("processed_data/h5ad_files/WT_LN_merged_scVI_processed.h5ad")
model = scvi.model.SCVI.load(
    "processed_data/scVI_models/WT_LN_merged",
    adata
)
# Get DE markers for cell type clusters using scVI differential expression method
# default thresholds for marker genes: lfc_mean > 0, bayes_factor > 3, non_zeros_proportion > 0.1
markers = rna.tl.cluster_de(
    adata,
    model,
    cluster_key="cell_type",
    save_path="analysis/DE_genes/WT_LN_merged/cell_type_markers/"
)

## Embeddings, all clusters

### Sample breakdown, all clusters

In [ ]:
rna.pl.sc_umap(
    adata,
    ["sample"],
    cmaps=["tab10"],
    legend_loc=[ "bottom"],
    latent_rep="X_tsne",
    ncols=1,
    legend_size=14,
    dims=(6,6),
    legend_fontsize=14,
    title_size=14,
    save_path="analysis/all_clusters_sample_source.pdf", 
    dpi=300,
)

### Sort breakdown, all clusters

In [ ]:
rna.pl.sc_umap(
    adata,
    ["sort"],
    cmaps=["tab10"],
    legend_loc=[ "bottom"],
    latent_rep="X_tsne",
    ncols=1,
    legend_size=14,
    dims=(6,6),
    legend_fontsize=14,
    title_size=14,
    save_path="analysis/all_clusters_sort.pdf", 
    dpi=300,
)

### Feature plots, all clusters

In [ ]:
expression = rna.tl.get_expression_matrix(adata, "scVI_normalized")
# Clip the bottom and top 1% of expression values for each gene to prevent outliers from affecting visualization
thresholds = [
    None,
    (expression["Aire"].quantile(0.01), expression["Aire"].quantile(0.99)),
    (expression["Rorc"].quantile(0.01), expression["Rorc"].quantile(0.99)),
    (expression["Itgav"].quantile(0.01), expression["Itgav"].quantile(0.99)),
    (expression["Itgb8"].quantile(0.01), expression["Itgb8"].quantile(0.99)),
    (expression["Ccr7"].quantile(0.01), expression["Ccr7"].quantile(0.99)),
    (expression["Cd80"].quantile(0.01), expression["Cd80"].quantile(0.99)),
    (expression["Cd86"].quantile(0.01), expression["Cd86"].quantile(0.99)),
    (expression["H2-Aa"].quantile(0.01), expression["H2-Aa"].quantile(0.99)),
    (expression["H2-Ab1"].quantile(0.01), expression["H2-Ab1"].quantile(0.99)),
    (expression["Klf4"].quantile(0.01), expression["Klf4"].quantile(0.99)),
    (expression["Myc"].quantile(0.01), expression["Myc"].quantile(0.99)),
    (expression["Ccr6"].quantile(0.01), expression["Ccr6"].quantile(0.99)),
    (expression["Cd40"].quantile(0.01), expression["Cd40"].quantile(0.99)),
    (expression["Cxcr6"].quantile(0.01), expression["Cxcr6"].quantile(0.99)),
    (expression["Il7r"].quantile(0.01), expression["Il7r"].quantile(0.99)),
    (expression["Thy1"].quantile(0.01), expression["Thy1"].quantile(0.99)),
    (expression["Cd44"].quantile(0.01), expression["Cd44"].quantile(0.99)),
    (expression["Il2ra"].quantile(0.01), expression["Il2ra"].quantile(0.99)),
]
rna.pl.sc_umap(
    adata,
    ["cell_type", "Aire", "Rorc", "Itgav", "Itgb8", "Ccr7", "Cd80", "Cd86", "H2-Aa", "H2-Ab1", "Klf4", "Myc", "Ccr6", "Cd40", "Cxcr6", "Il7r", "Thy1", "Cd44", "Il2ra"],
    cmaps=["tab20", "tab10", "viridis"],
    legend_loc=["None"],
    latent_rep="X_tsne",
    gene_data="scVI_normalized",
    ncols=3,
    dims=(6,6),
    legend_fontsize=21,
    title_size=21,
    save_path="analysis/WT_merged_feature_plots/littman_paper_all_clusters.png",
    cmap_label="Normalized expression",
    dpi=300,
    thresholds=thresholds,
)

## Cosine similarity, all clusters

In [ ]:
# Only need to run once to download immgen data, doesn't work on windows without installing wget
!wget "https://gist.github.com/vasilisNt/5e23eeefc188e1e772f428c74ef43277/raw/67f83d282b0b2180a8eeff74edf079d8826b12ba/immgen.tar.gz"
!tar -xzf immgen.tar.gz

In [ ]:
# Cell type similarity method uses the '.raw' attribute, make sure it contains raw counts (not log normalized
adataCopy = adata.copy()
del adataCopy.raw
adataCopy.X = adataCopy.layers["raw_counts"]
adataCopy.raw = adataCopy

# Add immgen similarity scores to adataCopy
adataCopy = rna.tl.cell_type_similarity(
    adataCopy,
    "immgen",
    save_path="analysis/all_WT_LN_similarity",
    dims=(6,6),
    legend_fontsize=21,
    title_size=21,
    cmap_label="Cosine similarity score",
)

In [ ]:
dataDict = {}
# Immgen samples of interest
keys = [
    'DC_8-_Th_imm_score',
    'DC_8+_Th_imm_score',
    'DC_4+_Sp_imm_score',
    'DC_8+_Sp_imm_score',
    'DC_8-4-11b-_Sp_imm_score',
    'DC_8-4-11b+_Sp_imm_score',
    'DC_4+_SLN_imm_score',
    'DC_8+_SLN_imm_score',
    'DC_8-4-11b-_SLN_imm_score',
    'DC_8-4-11b+_SLN_imm_score',
    'DC_4+_MLN_imm_score',
    'DC_8+_MLN_imm_score',
    'DC_8-4-11b-_MLN_imm_score',
    'DC_8-4-11b+_MLN_imm_score',
    'DC_103-11b+_Lv_imm_score',
    'DC_103+11b-_Lv_imm_score',
    'DC_103+11b-_LuLN_imm_score',
    'DC_103-11b+_LuLN_imm_score',
    'DC_103-11b+24+_Lu_imm_score',
    'DC_103+11b-_Lu_imm_score',
    'DC_103-11b+F4_imm_score',
    'DC_103+11b-_SI_imm_score',
    'DC_103+11b+_SI_imm_score',
    'DC_pDC_8-_Sp_imm_score',
    'DC_pDC_8+_Sp_imm_score',
    'DC_pDC_8+_SLN_imm_score',
    'DC_pDC_8+_MLN_imm_score',
    'DC_IIhilang-103-11blo_SLN_imm_score',
    'DC_IIhilang-103-11b+_SLN_imm_score',
    'DC_IIhilang+103+11blo_SLN_imm_score',
    'DC_IIhilang+103-11b+_SLN_imm_score',
    'DC_LC_Sk_imm_score',
    'Ep_MEChi_Th_imm_score',
    'Fi_MTS15+_Th_imm_score',
    'Fi_Sk_imm_score',
    'FRC_MLN_imm_score',
    'FRC_SLN_imm_score',
    'LEC_MLN_imm_score',
    'LEC_SLN_imm_score',
    'BEC_MLN_imm_score',
    'BEC_SLN_imm_score',
    'St_31-38-44-_SLN_imm_score',
    'proB_CLP_BM_imm_score',
    'proB_CLP_FL_imm_score',
    'proB_FrA_BM_imm_score',
    'proB_FrA_FL_imm_score',
    'proB_FrBC_BM_imm_score',
    'proB_FrBC_FL_imm_score',
    'preB_FrC_BM_imm_score',
    'preB_FrD_BM_imm_score',
    'preB_FrD_FL_imm_score',
    'B_FrE_BM_imm_score',
    'B_FrE_FL_imm_score',
    'B_T1_Sp_imm_score',
    'B_T2_Sp_imm_score',
    'B_T3_Sp_imm_score',
    'B_Fo_Sp_imm_score',
    'B_Fo_MLN_imm_score',
    'B_Fo_LN_imm_score',
    'B_Fo_PC_imm_score',
    'B_GC_Sp_imm_score',
    'B_MZ_Sp_imm_score',
    'B_FrF_BM_imm_score',
    'B1b_PC_imm_score',
    'B1a_PC_imm_score',
    'B1a_Sp_imm_score',
    'GN_BM_imm_score',
    'GN_Bl_imm_score',
    'MF_BM_imm_score',
    'MF_RP_Sp_imm_score',
    'MF_Lu_imm_score',
    'MF_103-11b+24-_Lu_imm_score',
    'MF_II+480lo_PC_imm_score',
    'MF_103-11b+_SI_imm_score',
    'MF_11cloSer_SI_imm_score',
    'MF_II-480hi_PC_imm_score',
    'MF_Microglia_CNS_imm_score',
    'Mo_6C+II-_BM_imm_score',
    'Mo_6C-II-_BM_imm_score',
    'Mo_6C+II-_Bl_imm_score',
    'Mo_6C+II+_Bl_imm_score',
    'Mo_6C-II-_Bl_imm_score',
    'Mo_6C-II+_Bl_imm_score',
    'Mo_6C-IIint_Bl_imm_score',
    'Mo_6C+II-_LN_imm_score',
    'NK_Sp_imm_score',
    'NK_49CI-_Sp_imm_score',
    'NK_49CI+_Sp_imm_score',
    'NK_49H-_Sp_imm_score',
    'NK_49H+_Sp_imm_score',
    'NK_b2m-_Sp_imm_score',
    'NK_DAP10-_Sp_imm_score',
    'NK_DAP12-_Sp_imm_score',
    'NK_CD127-_Sp_imm_score',
    'NK_CD49b+_Lv_imm_score',
    'NK_CD127-_SI_imm_score',
    'ILC1_CD127+_Sp_imm_score',
    'ILC1_CD49b-_Lv_imm_score',
    'ILC1_CD127+_SI_imm_score',
    'ILC2_SI_imm_score',
    'ILC3_NKp46+_Rorgthi_SI_imm_score',
    'ILC3_NKp46-_4-_SI_imm_score',
    'ILC3_NKp46-_4+_SI_imm_score',
    'preT_ETP_Th_imm_score',
    'preT_ETP-2A_Th_imm_score',
    'preT_DN2_Th_imm_score',
    'preT_DN2A_Th_imm_score',
    'preT_DN2B_Th_imm_score',
    'preT_DN2-3_Th_imm_score',
    'preT_DN3A_Th_imm_score',
    'preT_DN3B_Th_imm_score',
    'preT_DN3-4_Th_imm_score',
    'T_DN4_Th_imm_score',
    'T_ISP_Th_imm_score',
    'T_DP_Th_imm_score',
    'T_DPbl_Th_imm_score',
    'T_DPsm_Th_imm_score',
    'T_DP69+_Th_imm_score',
    'T_4+8int_Th_imm_score',
    'T_4SP69+_Th_imm_score',
    'T_4SP24int_Th_imm_score',
    'T_4SP24-_Th_imm_score',
    'T_4int8+_Th_imm_score',
    'T_8SP69+_Th_imm_score',
    'T_8SP24int_Th_imm_score',
    'T_8SP24-_Th_imm_score',
    'T_4Nve_Sp_imm_score',
    'T_4Mem_Sp_imm_score',
    'T_4Mem44h62l_Sp_imm_score',
    'T_4Nve_LN_imm_score',
    'T_4Mem_LN_imm_score',
    'T_4Mem44h62l_LN_imm_score',
    'T_4Nve_PP_imm_score',
    'T_4Nve_MLN_imm_score',
    'T_4_LN_BDC_imm_score',
    'T_4_PLN_BDC_imm_score',
    'T_4_Pa_BDC_imm_score',
    'T_4FP3-_Sp_imm_score',
    'T_4FP3+25+_Sp_imm_score',
    'T_4FP3+25+_AA_imm_score',
    'T_4FP3+25+_LN_imm_score',
    'T_8Nve_Sp_imm_score',
    'T_8Mem_Sp_imm_score',
    'T_8Nve_LN_imm_score',
    'T_8Mem_LN_imm_score',
    'T_8Nve_PP_imm_score',
    'T_8Nve_MLN_imm_score',
    'NKT_44-NK1_1-_Th_imm_score',
    'NKT_44+NK1_1-_Th_imm_score',
    'NKT_44+NK1_1+_Th_imm_score',
    'NKT_4+_Sp_imm_score',
    'NKT_4-_Sp_imm_score',
    'NKT_4+_Lv_imm_score',
    'NKT_4-_Lv_imm_score',
    'Tgd_Th_imm_score',
    'Tgd_vg1+vd6-24ahi_Th_imm_score',
    'Tgd_vg1+vd6+24ahi_Th_imm_score',
    'Tgd_vg2+24ahi_Th_imm_score',
    'Tgd_vg2+24ahi_e17_Th_imm_score',
    'Tgd_vg3+24ahi_e17_Th_imm_score',
    'Tgd_vg5+24ahi_Th_imm_score',
    'Tgd_vg1+vd6-24alo_Th_imm_score',
    'Tgd_vg1+vd6+24alo_Th_imm_score',
    'Tgd_vg2+24alo_Th_imm_score',
    'Tgd_vg3+24alo_e17_Th_imm_score',
    'Tgd_Sp_imm_score',
    'Tgd_vg2-_Sp_imm_score',
    'Tgd_vg2-_act_Sp_imm_score',
    'Tgd_vg2+_Sp_imm_score',
    'Tgd_vg2+_act_Sp_imm_score',
    'Tgd_vg2-_Sp_TCRbko_imm_score',
    'Tgd_vg2+_Sp_TCRbko_imm_score',
    'Tgd_vg5-_IEL_imm_score',
    'Tgd_vg5+_IEL_imm_score',
    'Tgd_vg5-_act_IEL_imm_score',
    'Tgd_vg5+_act_IEL_imm_score',
    ]
for key in keys:
    dataDict[key.rsplit("_", 2)[0]] = adataCopy.obs[key]
dataDict["cell_type"] = adataCopy.obs.cell_type
data = pd.DataFrame(dataDict)
cats = ["JC1", "JC2", "JC3", "mDC1-1", "mDC1-2", "mDC1-3", "mDC2-1", "mDC2-2", "mDC2-3", "mDC2-4", "mDC2-5", "cDC1", "cDC2", "pDC", "MF", "Mo", "BEC", "LEC", "Fibro1",
        "Fibro2", "Fibro3", "ILC", "ILC3", "CD4 T", "CD8 T", "B", "Plasma cell"]
data.cell_type.cat.reorder_categories(cats, inplace=True)
# Average similarity scores bycell type
data = data.groupby("cell_type").mean()
# Standardize scores to 0-1 for each immgen sample
data = data.subtract(data.min(axis=0), axis=1)
data = data.div(data.max(axis=0), axis=1)
# Clustered heatmap of data
ax = sns.clustermap(
    data,
    cmap="coolwarm",
    row_cluster=True,
    col_cluster=True,
    figsize=(65,16),
    cbar_pos=None,
)
ax.ax_heatmap.tick_params(axis="both", which="both", labelsize=21)
ax.ax_heatmap.set_ylabel("")
plt.tight_layout()
plt.savefig("analysis/all_cluster_similarity_heatmap_clustered.png", dpi=300)
plt.show()

## Subclustering

In [ ]:
# Subset on JCs, ILC3
adataSubset = adata[adata.obs.cell_type.isin(["JC1", "JC2", "JC3", "ILC3"])]
# Create new UMAP
sc.tl.umap(adataSubset, min_dist=0.5, spread=1)

### Embeddings

#### sample breakdown, subclusters

In [ ]:
rna.pl.sc_umap(
    adataSubset,
    ["sample"],
    cmaps=["tab10"],
    legend_loc=[ "bottom"],
    latent_rep="X_umap",
    ncols=1,
    legend_size=14,
    dims=(6,6),
    legend_fontsize=14,
    title_size=14,
    save_path="analysis/JC_subset_sample_source.pdf", 
    dpi=300,
)

#### Sort breakdown, subclusters

In [ ]:
rna.pl.sc_umap(
    adataSubset,
    ["sort"],
    cmaps=["tab10"],
    legend_loc=[ "bottom"],
    latent_rep="X_umap",
    ncols=1,
    legend_size=14,
    dims=(6,6),
    legend_fontsize=14,
    title_size=14,
    save_path="analysis/JC_subset_sort.pdf", 
    dpi=300,
)

#### Feature plots, subclusters

In [ ]:
rna.pl.sc_umap(
    adataSubset,
    ["cell_type", "Aire", "Rorc", "Itgav", "Itgb8", "Ccr7", "Cd80", "Cd86", "H2-Aa", "H2-Ab1", "Klf4", "Myc", "Ccr6", "Cd40", "Cxcr6", "Il7r", "Thy1", "Cd44", "Il2ra"],
    legend_loc=["None"],
    cmaps=["tab10"],
    gene_data="scVI_normalized",
    cmap_label="Normalized expression",
    title_size=21,
    legend_fontsize=21,
    dims=(6,6),
    s=6,
    save_path="analysis/WT_merged_feature_plots/littman_paper_JC_clusters.png",
    dpi=300,
    thresholds=thresholds, # Use same thresholds as for all cluster data so plots are comparable
)

### Dotplots

#### Features

In [ ]:
sc.set_figure_params(dpi=150, dpi_save=300, frameon=True)
sc.settings.figdir = "analysis/"
markers=["Rorc", "Itgav", "Itgb8", "Ccr7", "Cd80", "Cd86", "Aire", "H2-Aa", "H2-Ab1",]
sc.pl.dotplot(
    adataSubset, 
    markers, 
    groupby='cell_type', 
    dendrogram=False, 
    use_raw=False, # Use log normalized, scaled data
    swap_axes=True, 
    standard_scale="var", # Standardize each gene to range from 0-1
    categories_order=["JC1", "JC2", "JC3", "ILC3"],
    save="scaled_data_JC_subset_features.pdf"
)

#### TC1-4 genes

In [ ]:
sc.set_figure_params(dpi=150, dpi_save=300, frameon=True)
plt.rcdefaults()
sc.settings.figdir = "analysis/"
markersDict={"TC1": ["Nrp2", "Bach2", "Ncam1", "Nrxn1", "Cd86", "Cd80", "Nrg1", "Kif21a", "Tnfrsf11b"], #TC1
         "TC2": ["Pde10a", "Cfp", "Hk2", "Nrp1", "Wdfy4", "Ltb", "Pigr"], #TC2
         "TC3": ["Cd63", "Epsti1", "H2-M2", "Nlrc5", "Dnase1l3", "Ahcyl2", "Ldb2"], #TC3
         "TC4": ["Itgav", "Il15ra", "Il2ra", "Marcks", "Cd274", "Ccl5", "Cd36", "Itgb8", "Ccl22"], #TC4
            }
sc.pl.dotplot(
    adataSubset, 
    markersDict, 
    groupby='cell_type', 
    dendrogram=False, 
    use_raw=False, # Use log normalized, scaled data
    standard_scale="var", # Standardize each gene to range from 0-1
    categories_order=["JC1", "JC2", "JC3", "ILC3"],
    save="scaled_data_JC_subset_TC_markers.pdf",
)

### DE genes, subclusters

In [ ]:
model = scvi.model.SCVI.load(
    "processed_data/scVI_models/WT_LN_merged",
    adata
)
# Get DE markers for cell type clusters using scVI differential expression method
# default thresholds for marker genes: lfc_mean > 0, bayes_factor > 3, non_zeros_proportion > 0.1
markers = rna.tl.cluster_de(
    adata[adata.obs.cell_type.isin(["JC1", "JC2", "JC3", "ILC3"])], # Use only JC + ILC3 subclusters for DE
    model,
    cluster_key="cell_type",
    save_path="analysis/DE_genes/WT_LN_merged/cell_type_subset_markers/"
)
# Get top 15 DE genes for each cluster
JC1_de = markers["JC1"].head(15).index
JC2_de = markers["JC2"].head(15).index
JC3_de = markers["JC3"].head(15).index
ILC3_de = markers["ILC3"].head(15).index

In [ ]:
genes = list(JC1_de)
genes += [i for i in list(JC2_de) if i not in genes]
genes += [i for i in list(JC3_de) if i not in genes]
genes += [i for i in list(ILC3_de) if i not in genes]
adataSubset.obs.cell_type.cat.reorder_categories(["JC1", "JC2", "JC3", "ILC3"], inplace=True)
# Get normalized expression for DE genes
expression = rna.tl.get_expression_matrix(adataSubset, "scVI_normalized")
expression = expression.loc[:, genes]
expression["cell_type"] = adataSubset.obs.cell_type
# Pseudobulk average expression of DE genes by cell type
expression = expression.groupby("cell_type").mean()
# Clustered heatmap of data
ax = sns.clustermap(
    expression,
    cmap="viridis",
    z_score=1, # Z-score each gene
    row_cluster=False,
    col_cluster=False,
    cbar_pos=(1.001,0.18,0.03,0.64),
    figsize=(23,6)
)
cbar = ax.ax_heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)
ax.figure.axes[-1].set_ylabel("Z-score", size=21)
ax.ax_heatmap.tick_params(axis="both", which="both", labelsize=21)
ax.ax_heatmap.yaxis.tick_left()
ax.ax_heatmap.set_ylabel("")
plt.savefig("analysis/JC_DE_genes_pseudobulk.pdf", dpi=300, bbox_inches='tight')
plt.show()

### Cosine similarity, subclusters

In [ ]:
# Can only run after the all clusters similarity cells, uses the same data but subset on clusters of interest
dataDict = {}
# Immgen samples of interest
keys = [
    'DC_8-_Th_imm_score',
    'DC_8+_Th_imm_score',
    'DC_4+_Sp_imm_score',
    'DC_8+_Sp_imm_score',
    'DC_8-4-11b-_Sp_imm_score',
    'DC_8-4-11b+_Sp_imm_score',
    'DC_4+_SLN_imm_score',
    'DC_8+_SLN_imm_score',
    'DC_8-4-11b-_SLN_imm_score',
    'DC_8-4-11b+_SLN_imm_score',
    'DC_4+_MLN_imm_score',
    'DC_8+_MLN_imm_score',
    'DC_8-4-11b-_MLN_imm_score',
    'DC_8-4-11b+_MLN_imm_score',
    'DC_103-11b+_Lv_imm_score',
    'DC_103+11b-_Lv_imm_score',
    'DC_103+11b-_LuLN_imm_score',
    'DC_103-11b+_LuLN_imm_score',
    'DC_103-11b+24+_Lu_imm_score',
    'DC_103+11b-_Lu_imm_score',
    'DC_103-11b+F4_imm_score',
    'DC_103+11b-_SI_imm_score',
    'DC_103+11b+_SI_imm_score',
    'DC_pDC_8-_Sp_imm_score',
    'DC_pDC_8+_Sp_imm_score',
    'DC_pDC_8+_SLN_imm_score',
    'DC_pDC_8+_MLN_imm_score',
    'DC_IIhilang-103-11blo_SLN_imm_score',
    'DC_IIhilang-103-11b+_SLN_imm_score',
    'DC_IIhilang+103+11blo_SLN_imm_score',
    'DC_IIhilang+103-11b+_SLN_imm_score',
    'DC_LC_Sk_imm_score',
    'Ep_MEChi_Th_imm_score',
    'Fi_MTS15+_Th_imm_score',
    'Fi_Sk_imm_score',
    'FRC_MLN_imm_score',
    'FRC_SLN_imm_score',
    'LEC_MLN_imm_score',
    'LEC_SLN_imm_score',
    'BEC_MLN_imm_score',
    'BEC_SLN_imm_score',
    'St_31-38-44-_SLN_imm_score',
    'proB_CLP_BM_imm_score',
    'proB_CLP_FL_imm_score',
    'proB_FrA_BM_imm_score',
    'proB_FrA_FL_imm_score',
    'proB_FrBC_BM_imm_score',
    'proB_FrBC_FL_imm_score',
    'preB_FrC_BM_imm_score',
    'preB_FrD_BM_imm_score',
    'preB_FrD_FL_imm_score',
    'B_FrE_BM_imm_score',
    'B_FrE_FL_imm_score',
    'B_T1_Sp_imm_score',
    'B_T2_Sp_imm_score',
    'B_T3_Sp_imm_score',
    'B_Fo_Sp_imm_score',
    'B_Fo_MLN_imm_score',
    'B_Fo_LN_imm_score',
    'B_Fo_PC_imm_score',
    'B_GC_Sp_imm_score',
    'B_MZ_Sp_imm_score',
    'B_FrF_BM_imm_score',
    'B1b_PC_imm_score',
    'B1a_PC_imm_score',
    'B1a_Sp_imm_score',
    'GN_BM_imm_score',
    'GN_Bl_imm_score',
    'MF_BM_imm_score',
    'MF_RP_Sp_imm_score',
    'MF_Lu_imm_score',
    'MF_103-11b+24-_Lu_imm_score',
    'MF_II+480lo_PC_imm_score',
    'MF_103-11b+_SI_imm_score',
    'MF_11cloSer_SI_imm_score',
    'MF_II-480hi_PC_imm_score',
    'MF_Microglia_CNS_imm_score',
    'Mo_6C+II-_BM_imm_score',
    'Mo_6C-II-_BM_imm_score',
    'Mo_6C+II-_Bl_imm_score',
    'Mo_6C+II+_Bl_imm_score',
    'Mo_6C-II-_Bl_imm_score',
    'Mo_6C-II+_Bl_imm_score',
    'Mo_6C-IIint_Bl_imm_score',
    'Mo_6C+II-_LN_imm_score',
    'NK_Sp_imm_score',
    'NK_49CI-_Sp_imm_score',
    'NK_49CI+_Sp_imm_score',
    'NK_49H-_Sp_imm_score',
    'NK_49H+_Sp_imm_score',
    'NK_b2m-_Sp_imm_score',
    'NK_DAP10-_Sp_imm_score',
    'NK_DAP12-_Sp_imm_score',
    'NK_CD127-_Sp_imm_score',
    'NK_CD49b+_Lv_imm_score',
    'NK_CD127-_SI_imm_score',
    'ILC1_CD127+_Sp_imm_score',
    'ILC1_CD49b-_Lv_imm_score',
    'ILC1_CD127+_SI_imm_score',
    'ILC2_SI_imm_score',
    'ILC3_NKp46+_Rorgthi_SI_imm_score',
    'ILC3_NKp46-_4-_SI_imm_score',
    'ILC3_NKp46-_4+_SI_imm_score',
    'preT_ETP_Th_imm_score',
    'preT_ETP-2A_Th_imm_score',
    'preT_DN2_Th_imm_score',
    'preT_DN2A_Th_imm_score',
    'preT_DN2B_Th_imm_score',
    'preT_DN2-3_Th_imm_score',
    'preT_DN3A_Th_imm_score',
    'preT_DN3B_Th_imm_score',
    'preT_DN3-4_Th_imm_score',
    'T_DN4_Th_imm_score',
    'T_ISP_Th_imm_score',
    'T_DP_Th_imm_score',
    'T_DPbl_Th_imm_score',
    'T_DPsm_Th_imm_score',
    'T_DP69+_Th_imm_score',
    'T_4+8int_Th_imm_score',
    'T_4SP69+_Th_imm_score',
    'T_4SP24int_Th_imm_score',
    'T_4SP24-_Th_imm_score',
    'T_4int8+_Th_imm_score',
    'T_8SP69+_Th_imm_score',
    'T_8SP24int_Th_imm_score',
    'T_8SP24-_Th_imm_score',
    'T_4Nve_Sp_imm_score',
    'T_4Mem_Sp_imm_score',
    'T_4Mem44h62l_Sp_imm_score',
    'T_4Nve_LN_imm_score',
    'T_4Mem_LN_imm_score',
    'T_4Mem44h62l_LN_imm_score',
    'T_4Nve_PP_imm_score',
    'T_4Nve_MLN_imm_score',
    'T_4_LN_BDC_imm_score',
    'T_4_PLN_BDC_imm_score',
    'T_4_Pa_BDC_imm_score',
    'T_4FP3-_Sp_imm_score',
    'T_4FP3+25+_Sp_imm_score',
    'T_4FP3+25+_AA_imm_score',
    'T_4FP3+25+_LN_imm_score',
    'T_8Nve_Sp_imm_score',
    'T_8Mem_Sp_imm_score',
    'T_8Nve_LN_imm_score',
    'T_8Mem_LN_imm_score',
    'T_8Nve_PP_imm_score',
    'T_8Nve_MLN_imm_score',
    'NKT_44-NK1_1-_Th_imm_score',
    'NKT_44+NK1_1-_Th_imm_score',
    'NKT_44+NK1_1+_Th_imm_score',
    'NKT_4+_Sp_imm_score',
    'NKT_4-_Sp_imm_score',
    'NKT_4+_Lv_imm_score',
    'NKT_4-_Lv_imm_score',
    'Tgd_Th_imm_score',
    'Tgd_vg1+vd6-24ahi_Th_imm_score',
    'Tgd_vg1+vd6+24ahi_Th_imm_score',
    'Tgd_vg2+24ahi_Th_imm_score',
    'Tgd_vg2+24ahi_e17_Th_imm_score',
    'Tgd_vg3+24ahi_e17_Th_imm_score',
    'Tgd_vg5+24ahi_Th_imm_score',
    'Tgd_vg1+vd6-24alo_Th_imm_score',
    'Tgd_vg1+vd6+24alo_Th_imm_score',
    'Tgd_vg2+24alo_Th_imm_score',
    'Tgd_vg3+24alo_e17_Th_imm_score',
    'Tgd_Sp_imm_score',
    'Tgd_vg2-_Sp_imm_score',
    'Tgd_vg2-_act_Sp_imm_score',
    'Tgd_vg2+_Sp_imm_score',
    'Tgd_vg2+_act_Sp_imm_score',
    'Tgd_vg2-_Sp_TCRbko_imm_score',
    'Tgd_vg2+_Sp_TCRbko_imm_score',
    'Tgd_vg5-_IEL_imm_score',
    'Tgd_vg5+_IEL_imm_score',
    'Tgd_vg5-_act_IEL_imm_score',
    'Tgd_vg5+_act_IEL_imm_score',
    ]
for key in keys:
    dataDict[key.rsplit("_", 2)[0]] = adataCopy.obs[key]
dataDict["cell_type"] = adataCopy.obs.cell_type
data = pd.DataFrame(dataDict)
cats = ["JC1", "JC2", "JC3", "mDC1-1", "mDC1-2", "mDC1-3", "mDC2-1", "mDC2-2", "mDC2-3", "mDC2-4", "mDC2-5", "cDC1", "cDC2", "pDC", "MF", "Mo", "BEC", "LEC", "Fibro1",
        "Fibro2", "Fibro3", "ILC", "ILC3", "CD4 T", "CD8 T", "B", "Plasma cell"]
data.cell_type.cat.reorder_categories(cats, inplace=True)
# Get average similarity scores for each cell type
data = data.groupby("cell_type").mean()
# Subset on clusters of interest
data = data[data.index.isin(["JC1", "JC2", "JC3", "ILC3"])]
# Scale scores for each immgen sample to range from 0-1
data = data.subtract(data.min(axis=0), axis=1)
data = data.div(data.max(axis=0), axis=1)

# Clustered heatmap of scores
ax = sns.clustermap(
    data,
    cmap="coolwarm",
    row_cluster=False,
    col_cluster=True,
    figsize=(65,8),
    cbar_pos=None,
)
ax.ax_heatmap.tick_params(axis="both", which="both", labelsize=21)
ax.ax_heatmap.set_yticklabels(["JC1", "JC2", "JC3", "ILC3"], rotation=0, va="center")
ax.ax_heatmap.set_ylabel("")
ax.ax_heatmap.yaxis.tick_left()
plt.tight_layout()
plt.savefig("analysis/JC_cluster_similarity_heatmap_clustered.png", dpi=300)
plt.show()